# dataclasses 使用参考

In [1]:
# https://www.youtube.com/watch?v=vBH6GRJ1REM
import dataclasses
import inspect
from dataclasses import dataclass, field
from pprint import pprint
from typing import List
import attr

## 原始实现

In [2]:
class ManualComment:
    def __init__(self, id: int, text: str):
        self.id: int = id
        self.text: str = text

    def __repr__(self):
        return "{}(id={}, text={})".format(self.__class__.__name__, self.id, self.text)

    def __eq__(self, other):
        if other.__class__ is self.__class__:
            return (self.id, self.text) == (other.id, other.text)
        else:
            return NotImplemented

    def __ne__(self, other):
        result = self.__eq__(other)
        if result is NotImplemented:
            return NotImplemented
        else:
            return not result

    def __hash__(self):
        return hash((self.__class__, self.id, self.text))

    def __lt__(self, other):
        if other.__class__ is self.__class__: #类型安全，没有比较不同的类
            return (self.id, self.text) < (other.id, other.text)
        else:
            return NotImplemented

    def __le__(self, other):
        if other.__class__ is self.__class__:
            return (self.id, self.text) <= (other.id, other.text)
        else:
            return NotImplemented

    def __gt__(self, other):
        if other.__class__ is self.__class__:
            return (self.id, self.text) > (other.id, other.text)
        else:
            return NotImplemented

    def __ge__(self, other):
        if other.__class__ is self.__class__:
            return (self.id, self.text) >= (other.id, other.text)
        else:
            return NotImplemented

可能会将上面这个类放在字典中，因此要实现`__hash__`方法；同时既然可哈了，它的内容最好不可变：通过将数据成员前面添加两个下划线`self.__id`

同时为了支持比较，也有必要实现`__le__`&　`__gt__`等方法；functools 提供total_ordering 装饰器，可以只实现`__lt__`,其会自动补全剩下的方法。但是会比较慢，因此自己实现会比较好。

* ManualComment如果再增加一个数据成员 `author`，需要改动哪些地方？ 太复杂了。

##  dataclasses

1. 直接指明有哪些数据成员.通过field函数可以指定成员如何初始化、参与哪些行为, etc   
1. 可为dataclasses.dataclass dec传参的方式指定有哪些行为。是否可改变(frozen=True)，是否支持排序(order=True)

In [3]:
# frozen = True, immutable
# order = True, 

@dataclass(frozen=True, order=True)
class Comment:
    id: int
    text: str = "" # feild(default='')
    replies: List[int] = field(default_factory=list, repr=False, compare=False) # mutable default value?
        #compare=False, 不参与比较

In [6]:
comment = Comment(1, "I just subscribed!")
print(comment)

Comment(id=1, text='I just subscribed!')


In [12]:
# frozen=True
comment.id = 3  # can't immutable

FrozenInstanceError: cannot assign to field 'id'

In [15]:
c1 = Comment(20, 'hello')
c2 = Comment(30, 'jack')
c1 < c2

True

* default_factory=list #参数的默认参数不能是可变类型，否则会因为多个instance共享一个实例产生难以追踪的错误。因此采用构造工厂的方式，每次初始化都新建一个list

In [8]:
dataclasses.astuple(comment), dataclasses.asdict(comment)

((1, 'I just subscribed!', []),
 {'id': 1, 'text': 'I just subscribed!', 'replies': []})

In [17]:
inspect.getmembers(Comment, inspect.isfunction)

[('__delattr__',
  <function __main__.__create_fn__.<locals>.__delattr__(self, name)>),
 ('__eq__', <function __main__.__create_fn__.<locals>.__eq__(self, other)>),
 ('__ge__', <function __main__.__create_fn__.<locals>.__ge__(self, other)>),
 ('__gt__', <function __main__.__create_fn__.<locals>.__gt__(self, other)>),
 ('__hash__', <function __main__.__create_fn__.<locals>.__hash__(self)>),
 ('__init__',
  <function __main__.__create_fn__.<locals>.__init__(self, id: int, text: str = '', replies: List[int] = <factory>) -> None>),
 ('__le__', <function __main__.__create_fn__.<locals>.__le__(self, other)>),
 ('__lt__', <function __main__.__create_fn__.<locals>.__lt__(self, other)>),
 ('__repr__', <function __main__.__create_fn__.<locals>.__repr__(self)>),
 ('__setattr__',
  <function __main__.__create_fn__.<locals>.__setattr__(self, name, value)>)]

In [ ]:
# slots=True 什么意思？